In [1]:
import warnings
import pandas as pd
import numpy as np
import yfinance as yf
import pandas_ta as ta
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.renderers.default = 'iframe'

# Suprimir advertencias FutureWarning
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
def download_data(start, end, ticker):
    return yf.download(ticker, start=start, end=end)

In [3]:
def calculate_signals(data, indicator):
    if indicator == 'SMA':
        # Calcular medias móviles simples usando pandas_ta
        data['SMA_50'] = ta.sma(data['Close'], length=50)
        data['SMA_200'] = ta.sma(data['Close'], length=200)
        
        # Señales de cruce de medias móviles simples
        data['Signal_SMA'] = np.where(data['SMA_50'] > data['SMA_200'], 1, 0)
        data['Position'] = data['Signal_SMA'].diff()
        data['Position'] = np.where(data['Position'] == 1, 1, np.where(data['Position'] == -1, -1, 0))
        
    elif indicator == 'EMA':
        # Calcular medias móviles exponenciales usando pandas_ta
        data['EMA_50'] = ta.ema(data['Close'], length=50)
        data['EMA_200'] = ta.ema(data['Close'], length=200)
        
        # Señales de cruce de medias móviles exponenciales
        data['Signal_EMA'] = np.where(data['EMA_50'] > data['EMA_200'], 1, 0)
        data['Position'] = data['Signal_EMA'].diff()
        data['Position'] = np.where(data['Position'] == 1, 1, np.where(data['Position'] == -1, -1, 0))

    elif indicator == 'RSI':
        # Calcular RSI
        data['RSI'] = ta.rsi(data['Close'], length=14)
        
        # Señales de RSI
        data['RSI_Signal'] = 0
        data['RSI_Signal'] = np.where(data['RSI'] < 30, 1, data['RSI_Signal'])  # Señal de compra
        data['RSI_Signal'] = np.where(data['RSI'] > 70, -1, data['RSI_Signal'])  # Señal de venta
        data['Position'] = data['RSI_Signal']

    elif indicator == 'MACD':
        # Calcular MACD
        data['MACD'] = ta.macd(data['Close'], fast=12, slow=26, signal=9)[0]
        data['MACD_Signal'] = ta.macd(data['Close'], fast=12, slow=26, signal=9)[1]
        
        # Señales de MACD
        data['MACD_Signal_Cross'] = np.where(data['MACD'] > data['MACD_Signal'], 1, 0)  # Señal de compra
        data['MACD_Signal_Cross'] = np.where(data['MACD'] < data['MACD_Signal'], -1, data['MACD_Signal_Cross'])  # Señal de venta
        data['Position'] = data['MACD_Signal_Cross']

    elif indicator == 'Bollinger':        
        # Calcular Bandas de Bollinger
        bbands_result = ta.bbands(data['Close'], length=20, std=2)

        # Asignar las columnas al DataFrame 'data' según sea necesario
        data['Bollinger_Upper'] = bbands_result['BBU_20_2.0']
        data['Bollinger_Middle'] = bbands_result['BBM_20_2.0']
        data['Bollinger_Lower'] = bbands_result['BBL_20_2.0']

        # Opcionalmente, si necesitas otras columnas como el ancho y la desviación estándar
        data['Bollinger_width'] = bbands_result['BBB_20_2.0']
        data['Bollinger_std'] = bbands_result['BBP_20_2.0']
        
        # Señales de Bandas de Bollinger
        data['Bollinger_Signal'] = 0
        data['Bollinger_Signal'] = np.where(data['Close'] < data['Bollinger_Lower'], 1, data['Bollinger_Signal'])  # Señal de compra
        data['Bollinger_Signal'] = np.where(data['Close'] > data['Bollinger_Upper'], -1, data['Bollinger_Signal'])  # Señal de venta
        data['Position'] = data['Bollinger_Signal']

    elif indicator == 'Stochastic':
        # Calcular Oscilador Estocástico
        sto_result = ta.stoch(data['High'], data['Low'], data['Close'], k=14, d=3, smooth_k=3)
        
        # Asignar las columnas al DataFrame 'data' según sea necesario
        data['Stoch_K'] = sto_result['STOCHk_14_3_3']
        data['Stoch_D'] = sto_result['STOCHd_14_3_3']
        
        # Señales del Oscilador Estocástico
        data['Stoch_Signal'] = 0
        data['Stoch_Signal'] = np.where((data['Stoch_K'] < 20) & (data['Stoch_D'] < 20), 1, data['Stoch_Signal'])  # Señal de compra
        data['Stoch_Signal'] = np.where((data['Stoch_K'] > 80) & (data['Stoch_D'] > 80), -1, data['Stoch_Signal'])  # Señal de venta
        data['Position'] = data['Stoch_Signal']

    elif indicator == 'Volume':
        # Calcular Media Móvil de Volumen
        data['Volume_SMA_20'] = ta.sma(data['Volume'], length=20)
        
        # Señales de Volumen
        data['Volume_Signal'] = 0
        data['Volume_Signal'] = np.where(data['Volume'] > data['Volume_SMA_20'], 1, data['Volume_Signal'])  # Señal de compra
        data['Volume_Signal'] = np.where(data['Volume'] < data['Volume_SMA_20'], -1, data['Volume_Signal'])  # Señal de venta
        data['Position'] = data['Volume_Signal']

    else:
        raise ValueError("El parámetro 'indicator' debe ser 'SMA', 'EMA', 'RSI', 'MACD', 'Bollinger', 'Stochastic' o 'Volume'.")
    
    return data


In [4]:
def simulate_backtesting(data, initial_capital, purchase_fraction, sell_fraction, position_column):
    data['Portfolio Value'] = initial_capital
    data['Holdings'] = 0.0
    data['Cash'] = initial_capital
    data['Shares Owned'] = 0.0

    for i in range(1, len(data)):
        if data[position_column][i] == 1:  # Comprar
            amount_to_spend = data['Cash'].iloc[i-1] * purchase_fraction
            shares_bought = amount_to_spend / data['Close'][i]
            data.loc[data.index[i], 'Holdings'] = data['Holdings'].iloc[i-1] + shares_bought
            data.loc[data.index[i], 'Cash'] = data['Cash'].iloc[i-1] - shares_bought * data['Close'][i]
        elif data[position_column][i] == -1:  # Vender
            shares_to_sell = data['Holdings'].iloc[i-1] * sell_fraction
            data.loc[data.index[i], 'Cash'] = data['Cash'].iloc[i-1] + shares_to_sell * data['Close'][i]
            data.loc[data.index[i], 'Holdings'] = data['Holdings'].iloc[i-1] - shares_to_sell
        else:  # Mantener
            data.loc[data.index[i], 'Holdings'] = data['Holdings'].iloc[i-1]
            data.loc[data.index[i], 'Cash'] = data['Cash'].iloc[i-1]

        data.loc[data.index[i], 'Portfolio Value'] = data['Cash'][i] + data['Holdings'][i] * data['Close'][i]
        data.loc[data.index[i], 'Shares Owned'] = data['Holdings'][i]

    final_portfolio_value = data['Portfolio Value'].iloc[-1]
    total_return = (final_portfolio_value - initial_capital) / initial_capital * 100
    return final_portfolio_value, total_return, data

In [5]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def plot_results(data, indicator, ticker):
    # Asegurarse de que no hay NaN en las columnas que se van a plotear
    required_columns = ['Close', 'Portfolio Value', 'Shares Owned']
    
    # Añadir columnas específicas para cada indicador según las señales generadas
    if indicator in ['SMA', 'EMA']:
        required_columns += [f'{indicator}_50', f'{indicator}_200', 'Position']
    elif indicator == 'RSI':
        required_columns += ['RSI', 'Position']
    elif indicator == 'MACD':
        required_columns += ['MACD', 'MACD_Signal_Cross', 'Position']
    elif indicator == 'Bollinger':
        required_columns += ['Bollinger_Upper', 'Bollinger_Middle', 'Bollinger_Lower', 'Position']
    elif indicator == 'Stochastic':
        required_columns += ['Stoch_K', 'Stoch_D', 'Position']
    elif indicator == 'Volume':
        required_columns += ['Volume', 'Volume_Signal', 'Position']
    
    # Eliminar filas con NaN en columnas requeridas
    data = data.dropna(subset=required_columns)

    # Visualización de resultados con Plotly
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.1,
                        row_heights=[1, 1],
                        specs=[[{"secondary_y": True}]] * 2)  # Especificación para todos los subgráficos con secondary_y=True

    # Gráfico de precios y indicador seleccionado
    fig.add_trace(go.Scatter(x=data.index, y=data['Close'], mode='lines', name='Precio de Cierre', line=dict(color='blue')), row=1, col=1, secondary_y=False)
    
    # Añadir gráficos específicos para cada indicador
    if indicator in ['SMA', 'EMA']:
        fig.add_trace(go.Scatter(x=data.index, y=data[f'{indicator}_50'], mode='lines', name=f'{indicator} de 50 días', line=dict(color='orange')), row=1, col=1, secondary_y=True)
        fig.add_trace(go.Scatter(x=data.index, y=data[f'{indicator}_200'], mode='lines', name=f'{indicator} de 200 días', line=dict(color='green')), row=1, col=1, secondary_y=True)
        
    elif indicator == 'RSI':
        fig.add_trace(go.Scatter(x=data.index, y=data['RSI'], mode='lines', name='RSI', line=dict(color='purple')), row=1, col=1, secondary_y=True)
        
    elif indicator == 'MACD':
        fig.add_trace(go.Scatter(x=data.index, y=data['MACD'], mode='lines', name='MACD', line=dict(color='red')), row=1, col=1, secondary_y=True)
        fig.add_trace(go.Scatter(x=data.index, y=data['MACD_Signal_Cross'], mode='lines', name='MACD Signal', line=dict(color='blue')), row=1, col=1, secondary_y=True)
        
    elif indicator == 'Bollinger':
        fig.add_trace(go.Scatter(x=data.index, y=data['Bollinger_Upper'], mode='lines', name='Bollinger Upper', line=dict(color='orange')), row=1, col=1, secondary_y=True)
        fig.add_trace(go.Scatter(x=data.index, y=data['Bollinger_Middle'], mode='lines', name='Bollinger Middle', line=dict(color='purple')), row=1, col=1, secondary_y=True)
        fig.add_trace(go.Scatter(x=data.index, y=data['Bollinger_Lower'], mode='lines', name='Bollinger Lower', line=dict(color='green')), row=1, col=1, secondary_y=True)
        
    elif indicator == 'Stochastic':
        fig.add_trace(go.Scatter(x=data.index, y=data['Stoch_K'], mode='lines', name='Stochastic K', line=dict(color='orange')), row=1, col=1, secondary_y=True)
        fig.add_trace(go.Scatter(x=data.index, y=data['Stoch_D'], mode='lines', name='Stochastic D', line=dict(color='purple')), row=1, col=1, secondary_y=True)
        
    elif indicator == 'Volume':
        fig.add_trace(go.Scatter(x=data.index, y=data['Volume'], mode='lines', name='Volumen', line=dict(color='green')), row=1, col=1, secondary_y=True)
        
    # Señales de compra y venta para el indicador seleccionado
    if 'Position' in data.columns:
        if indicator in ['SMA', 'EMA']:
            fig.add_trace(go.Scatter(x=data[data['Position'] == 1].index, y=data[f'{indicator}_50'][data['Position'] == 1], mode='markers', marker_symbol='triangle-up', marker_color='green', marker_size=10, name=f'Señal de Compra ({indicator})'), row=1, col=1)
            fig.add_trace(go.Scatter(x=data[data['Position'] == -1].index, y=data[f'{indicator}_50'][data['Position'] == -1], mode='markers', marker_symbol='triangle-down', marker_color='red', marker_size=10, name=f'Señal de Venta ({indicator})'), row=1, col=1)
        else:
            # Para otros indicadores donde no se usa {indicator}_50
            fig.add_trace(go.Scatter(x=data[data['Position'] == 1].index, y=data['Close'][data['Position'] == 1], mode='markers', marker_symbol='triangle-up', marker_color='green', marker_size=10, name=f'Señal de Compra ({indicator})'), row=1, col=1)
            fig.add_trace(go.Scatter(x=data[data['Position'] == -1].index, y=data['Close'][data['Position'] == -1], mode='markers', marker_symbol='triangle-down', marker_color='red', marker_size=10, name=f'Señal de Venta ({indicator})'), row=1, col=1)

    # Gráfico del valor de la cartera y acciones poseídas para el indicador seleccionado
    fig.add_trace(go.Scatter(x=data.index, y=data['Portfolio Value'], mode='lines', name=f'Valor de la Cartera ({indicator})', line=dict(color='purple')), row=2, col=1, secondary_y=False)
    fig.add_trace(go.Scatter(x=data.index, y=data['Shares Owned'], mode='lines', name=f'Acciones Poseídas ({indicator})', line=dict(color='black')), row=2, col=1, secondary_y=True)

    # Actualizar diseño del gráfico
    fig.update_layout(title=f'Estrategia de Trading de {ticker} con {indicator}', xaxis_title='Fecha', template='plotly_white')
    fig.update_yaxes(title_text='Precio', row=1, col=1, secondary_y=False)
    fig.update_yaxes(title_text='Indicador', row=1, col=1, secondary_y=True)
    fig.update_yaxes(title_text='Valor de la Cartera', row=2, col=1, secondary_y=False)
    fig.update_yaxes(title_text='Acciones Poseídas', row=2, col=1, secondary_y=True)

    fig.show()


In [6]:
def backtest_strategy(start, end,initial_capital, ticker, purchase_fraction, sell_fraction, indicator):
    # Descargar datos y calcular señales
    data = download_data(start, end, ticker)
    data = calculate_signals(data.copy(), indicator=indicator)

    # Simulación
    final_value, total_return, data_backtesting = simulate_backtesting(data.copy(),
                                                                       initial_capital,
                                                                       purchase_fraction,
                                                                       sell_fraction,
                                                                       'Position')

    # Plotear resultados
    plot_results(data_backtesting, indicator, ticker)

    return final_value, total_return

In [7]:
'''
'SMA': Medias Móviles Simples.
'EMA': Medias Móviles Exponenciales.
'RSI': Índice de Fuerza Relativa.
'MACD': Convergencia/Divergencia de Medias Móviles.
'Bollinger': Bandas de Bollinger.
'Stochastic': Oscilador Estocástico.
'Volume': Volumen.
'''

start= '2023-01-01'
end= '2024-01-01'
initial_capital = 1000
purchase_fraction = 0.80  # 60% del capital para compras
sell_fraction = 0.60  # 60% del capital para ventas
ticker = 'AAPL'
indicator='Stochastic'

final_value, total_return = backtest_strategy(start,
                                              end,
                                              initial_capital,
                                              ticker,
                                              purchase_fraction,
                                              sell_fraction,
                                              indicator)

print(f'Valor final de la cartera ({indicator}): ${final_value:.2f}')
print(f'Retorno total ({indicator}): {total_return:.2f}%')

[*********************100%%**********************]  1 of 1 completed


Valor final de la cartera (Stochastic): $1256.47
Retorno total (Stochastic): 25.65%


In [9]:
# Lista de todos los indicadores a probar
indicadores = ['SMA', 'EMA', 'RSI', 'MACD', 'Bollinger', 'Stochastic', 'Volume']

# Parámetros comunes para el backtest
start= '2023-01-01'
end= '2024-01-01'
initial_capital = 1000
purchase_fraction = 0.80  # 80% del capital para compras
sell_fraction = 0.60  # 60% del capital para ventas
ticker = 'AAPL'

# Bucle para probar todos los indicadores
for indicator in indicadores:
    try:
        # Ejecutar el backtest para el indicador actual
        final_value, total_return = backtest_strategy(start,
                                                      end,
                                                      initial_capital,
                                                      ticker,
                                                      purchase_fraction,
                                                      sell_fraction,
                                                      indicator)

        # Imprimir los resultados del backtest para el indicador actual
        print(f'Valor final de la cartera ({indicator}): ${final_value:.2f}')
        print(f'Retorno total ({indicator}): {total_return:.2f}%')
        print()  # Salto de línea para separar los resultados de diferentes indicadores

    except Exception as e:
        print(f'Ocurrió un error al procesar el indicador {indicator}: {str(e)}')
        print(f'Saltando al siguiente indicador...\n')


[*********************100%%**********************]  1 of 1 completed


[*********************100%%**********************]  1 of 1 completed

Valor final de la cartera (SMA): $1075.93
Retorno total (SMA): 7.59%



[*********************100%%**********************]  1 of 1 completed

Valor final de la cartera (EMA): $1075.93
Retorno total (EMA): 7.59%



...

[*********************100%%**********************]  1 of 1 completed

Valor final de la cartera (Bollinger): $1138.09
Retorno total (Bollinger): 13.81%



[*********************100%%**********************]  1 of 1 completed

Valor final de la cartera (Stochastic): $1256.47
Retorno total (Stochastic): 25.65%



Valor final de la cartera (Volume): $1118.54
Retorno total (Volume): 11.85%

